In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import utils
import time
import pickle

from keras.models import Model, Input
from keras.layers import Dense, Conv1D, BatchNormalization, GlobalMaxPooling1D, Dropout, \
    Embedding, Concatenate, SpatialDropout1D, MaxPooling1D
from keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score

from utils.preprocessing_utils import tokenize_sentences, convert_tokens_to_padded_sequence, \
    remove_punctuation, remove_punctuation_weak, perform_stemming, perform_lemmatization, replace_unknown_tokens
from utils.dataset_utils import load_data_from_csv
from utils.embedding_utils import create_embeddings_mapping, load_word2vec_embeddings
from utils.training_utils import train_and_evaluate_model
from utils.keras_utils import DataSet

In [2]:
import importlib
import utils
importlib.reload(utils.embedding_utils)
importlib.reload(utils.dataset_utils)
importlib.reload(utils.preprocessing_utils)
importlib.reload(utils.training_utils)
importlib.reload(utils.keras_utils)

<module 'utils.keras_utils' from '/home/philipp/work/gitprojects/toxic-comment-experiments/utils/keras_utils.py'>

In [2]:
random_seed = 2018
np.random.seed(random_seed)

trained_weights = 'data/models/multiwindow_model.hdf5'
embeddings_path = 'data/models/transferlearning/embeddings'
embedding_length = 300

youtoxic_classes = ['IsToxic', 'IsAbusive', 'IsHatespeech']
youtoxic_features = 'Text'
path_youtoxic_data = 'data/youtoxic/youtoxic.csv'
yt_epochs = 20

kaggle_classes = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
kaggle_features = 'comment_text'
path_kaggle_train_data = 'data/kaggle/train.csv'
path_kaggle_test_data = 'data/kaggle/test_complete.csv'

batch_size = 64
epochs = 5

In [3]:
def convert_tokens_to_padded_indices(X_train_tok, X_test_tok, word_embedding_mapping):
    max_len_train = X_train_tok.apply(lambda x: len(x)).max()
    max_len_test = X_test_tok.apply(lambda x: len(x)).max()
    # limit length to 2000, otherwise we get a MemoryError
    print("maximum comment length: {}".format(max(max_len_train, max_len_test)))
    max_comment_length = min(2000, max(max_len_train, max_len_test))
    X_train_input = convert_tokens_to_padded_sequence(X_train_tok, word_embedding_mapping, max_comment_length)
    X_test_input = convert_tokens_to_padded_sequence(X_test_tok, word_embedding_mapping, max_comment_length)
    return X_train_input, X_test_input, max_comment_length

def convert_all_tokens_to_padded_indices(X_tok, word_embedding_mapping):
    max_len = X_tok.apply(lambda x: len(x)).max()    
    # limit length to 2000, otherwise we get a MemoryError
    print("maximum comment length: {}".format(max_len))
    max_comment_length = min(2000, max_len)
    X_input = convert_tokens_to_padded_sequence(X_tok, word_embedding_mapping, max_comment_length)
    return X_input, max_comment_length
    
def generate_model(num_tokens, max_comment_length, classes):
    # hyperparameters
    kernel_sizes = [3, 4, 5]
    hidden_dim = 100
    num_filters = [100, 100, 100]
    dropout = 0.4
    spatial_dropout = 0.2
    
    # model
    input = Input((max_comment_length,))
    word_emb = Embedding(input_dim=num_tokens, output_dim=embedding_length, input_length=max_comment_length)(input)
    word_emb = SpatialDropout1D(spatial_dropout)(word_emb)

    conv1 = Conv1D(kernel_size=kernel_sizes[0], filters=num_filters[0], padding='same')(word_emb)
    conv1 = GlobalMaxPooling1D()(conv1)

    conv2 = Conv1D(kernel_size=kernel_sizes[1], filters=num_filters[1], padding='same')(word_emb)
    conv2 = GlobalMaxPooling1D()(conv2)

    conv3 = Conv1D(kernel_size=kernel_sizes[2], filters=num_filters[2], padding='same')(word_emb)
    conv3 = GlobalMaxPooling1D()(conv3)

    concat4 = Concatenate()([conv1, conv2, conv3])

    fc5 = Dense(hidden_dim, activation='relu')(concat4)
    fc5 = Dropout(dropout)(fc5)
    output = Dense(len(classes), activation='sigmoid')(fc5)

    model = Model(inputs=[input], outputs=[output])
    return model, fc5

### Train model on kaggle data

In [6]:
X_train_kaggle, Y_train_kaggle = load_data_from_csv(path_kaggle_train_data, kaggle_features, kaggle_classes)
X_test_kaggle, Y_test_kaggle = load_data_from_csv(path_kaggle_test_data, kaggle_features, kaggle_classes)

In [7]:
X_train_tok_kaggle = tokenize_sentences(X_train_kaggle)
X_test_tok_kaggle = tokenize_sentences(X_test_kaggle)

In [8]:
embeddings_mapping = create_embeddings_mapping(X_train_tok_kaggle, X_test_tok_kaggle, debug=True)
X_train_input_kaggle, X_test_input_kaggle, max_comment_length = \
    convert_tokens_to_padded_indices(X_train_tok_kaggle, X_test_tok_kaggle, embeddings_mapping)
del X_train_tok_kaggle
del X_test_tok_kaggle

Number of unique tokens: 326175
maximum comment length: 3801


In [13]:
model, _ = generate_model(len(embeddings_mapping)+1, max_comment_length, kaggle_classes)
scores = train_and_evaluate_model(model, X_train_input_kaggle, Y_train_kaggle, (X_test_input_kaggle, Y_test_kaggle), \
                                    epochs, batch_size, 'adam', 'binary_crossentropy', ['accuracy'], \
                                    runs=1, weights_path=trained_weights, write_model_checkpoints=True)

RUN 1/1
Train on 159571 samples, validate on 63978 samples
Epoch 1/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0604 - acc: 0.9791

/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 train: ROC-AUC - epoch: 1 - score: 0.99365
 Tox: 0.99121 - STox: 0.99154 - Obs: 0.99543 - Thr: 0.98418 - Ins: 0.99084 - IdH: 0.98343
 train: F1 Score - epoch: 1 - score: 0.77600
 Tox: 0.83246 - STox: 0.33201 - Obs: 0.84516 - Thr: 0.00000 - Ins: 0.76027 - IdH: 0.00000


/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 val: ROC-AUC - epoch: 1 - score: 0.97771
 Tox: 0.96220 - STox: 0.98701 - Obs: 0.97512 - Thr: 0.97296 - Ins: 0.96945 - IdH: 0.96560
 val: F1 Score - epoch: 1 - score: 0.64002
 Tox: 0.67963 - STox: 0.34385 - Obs: 0.66947 - Thr: 0.00000 - Ins: 0.63622 - IdH: 0.00000

Epoch 00001: val_loss improved from inf to 0.07313, saving model to data/models/multiwindow_model.hdf5
159571/159571 [==============================] - 679s 4ms/step - loss: 0.0604 - acc: 0.9791 - val_loss: 0.0731 - val_acc: 0.9712
Epoch 2/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0387 - acc: 0.9847
 train: ROC-AUC - epoch: 2 - score: 0.99696
 Tox: 0.99656 - STox: 0.99351 - Obs: 0.99751 - Thr: 0.99510 - Ins: 0.99482 - IdH: 0.99500
 train: F1 Score - epoch: 2 - score: 0.82195
 Tox: 0.87447 - STox: 0.00000 - Obs: 0.88741 - Thr: 0.07600 - Ins: 0.82117 - IdH: 0.26481


/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 val: ROC-AUC - epoch: 2 - score: 0.97975
 Tox: 0.96164 - STox: 0.98588 - Obs: 0.97768 - Thr: 0.98178 - Ins: 0.97230 - IdH: 0.97321
 val: F1 Score - epoch: 2 - score: 0.64778
 Tox: 0.66534 - STox: 0.00000 - Obs: 0.69136 - Thr: 0.11207 - Ins: 0.65724 - IdH: 0.28604

Epoch 00002: val_loss improved from 0.07313 to 0.06797, saving model to data/models/multiwindow_model.hdf5
159571/159571 [==============================] - 680s 4ms/step - loss: 0.0387 - acc: 0.9847 - val_loss: 0.0680 - val_acc: 0.9718
Epoch 3/5
159552/159571 [============================>.] - ETA: 0s - loss: 0.0298 - acc: 0.9879
 train: ROC-AUC - epoch: 3 - score: 0.99831
 Tox: 0.99820 - STox: 0.99579 - Obs: 0.99870 - Thr: 0.99804 - Ins: 0.99671 - IdH: 0.99717
 train: F1 Score - epoch: 3 - score: 0.87703
 Tox: 0.92944 - STox: 0.37670 - Obs: 0.90498 - Thr: 0.38523 - Ins: 0.86458 - IdH: 0.67011

 val: ROC-AUC - epoch: 3 - score: 0.97804
 Tox: 0.96037 - STox: 0.98458 - Obs: 0.97407 - Thr: 0.98383 - Ins: 0.96933 - IdH: 0.97360

In [10]:
with open(embeddings_path, 'wb') as file:
    pickle.dump(embeddings_mapping, file, protocol=pickle.HIGHEST_PROTOCOL)

### Transfer learning to YouToxic data set

In [5]:
X_youtoxic, Y_youtoxic = load_data_from_csv(path_youtoxic_data, youtoxic_features, youtoxic_classes)

X_tok_youtoxic = tokenize_sentences(X_youtoxic)

In [6]:
with open(embeddings_path, 'rb')  as file:
    embeddings_matrix = pickle.load(file)
    print(len(embeddings_matrix))

326175


In [7]:
X_input_youtoxic, max_comment_length = \
    convert_all_tokens_to_padded_indices(X_tok_youtoxic, embeddings_matrix)

maximum comment length: 969


Load trained kaggle model and cut off last layer. Then add a new output layer for the labels of the youtoxic data set.

In [10]:
loaded_model, fc5 = generate_model(max_comment_length=max_comment_length, classes=kaggle_classes, num_tokens=len(embeddings_matrix)+1)
loaded_model.load_weights(trained_weights)

new_output = Dense(len(youtoxic_classes), activation='sigmoid')(fc5)
yt_model = Model(inputs=loaded_model.inputs, outputs=[new_output])

In [14]:
run_scores = []
for run in range(5):
    fold_scores = []
    kfold = KFold(5, shuffle=True)
    for train_index, test_index in kfold.split(X_input_youtoxic, Y_youtoxic):    
        X_train, X_test = X_input_youtoxic[train_index], X_input_youtoxic[test_index]
        y_train, y_test = Y_youtoxic.values[train_index], Y_youtoxic.values[test_index]
        scores = train_and_evaluate_model(yt_model, X_train, y_train, (X_test, y_test), \
                                 yt_epochs, batch_size, 'adam', 'binary_crossentropy', ['accuracy'], \
                                 seed=random_seed, runs=1, dataset=DataSet.YOUTOXIC)
        fold_scores.append(scores[0,:])
    fold_scores_matrix = np.array(fold_scores).mean(axis=0)
    run_scores.append(fold_scores_matrix)
run_scores_matrix = np.array(run_scores)

RUN 1/1
Train on 800 samples, validate on 200 samples
Epoch 1/20
768/800 [===========================>..] - ETA: 0s - loss: 0.6329 - acc: 0.6280
 train: ROC-AUC - epoch: 1 - score: 0.74653
 IsToxic: 0.87287 - IsAbusive: 0.75061 - IsHatespeech: 0.46771
 train: F1 Score - epoch: 1 - score: 0.00000
 sToxic: 0.00000 - IsAbusive: 0.00000 - IsHatespeech: 0.00000

 val: ROC-AUC - epoch: 1 - score: 0.65778
 IsToxic: 0.56652 - IsAbusive: 0.60144 - IsHatespeech: 0.43701
 val: F1 Score - epoch: 1 - score: 0.00000
 sToxic: 0.00000 - IsAbusive: 0.00000 - IsHatespeech: 0.00000
800/800 [==============================] - 3s 3ms/step - loss: 0.6309 - acc: 0.6304 - val_loss: 0.5743 - val_acc: 0.7167


/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'pre

Epoch 2/20
768/800 [===========================>..] - ETA: 0s - loss: 0.5939 - acc: 0.6719
 train: ROC-AUC - epoch: 2 - score: 0.79040
 IsToxic: 0.96541 - IsAbusive: 0.95486 - IsHatespeech: 0.60117
 train: F1 Score - epoch: 2 - score: 0.55755
 sToxic: 0.79872 - IsAbusive: 0.00000 - IsHatespeech: 0.00000

 val: ROC-AUC - epoch: 2 - score: 0.67031
 IsToxic: 0.58959 - IsAbusive: 0.67510 - IsHatespeech: 0.45457
 val: F1 Score - epoch: 2 - score: 0.45326
 sToxic: 0.60150 - IsAbusive: 0.00000 - IsHatespeech: 0.00000
800/800 [==============================] - 2s 3ms/step - loss: 0.5921 - acc: 0.6738 - val_loss: 0.5856 - val_acc: 0.6783


/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 3/20
768/800 [===========================>..] - ETA: 0s - loss: 0.5746 - acc: 0.6910
 train: ROC-AUC - epoch: 3 - score: 0.81275
 IsToxic: 0.98993 - IsAbusive: 0.95237 - IsHatespeech: 0.85130
 train: F1 Score - epoch: 3 - score: 0.06667
 sToxic: 0.10973 - IsAbusive: 0.03311 - IsHatespeech: 0.00000

 val: ROC-AUC - epoch: 3 - score: 0.68932
 IsToxic: 0.68052 - IsAbusive: 0.68142 - IsHatespeech: 0.51985
 val: F1 Score - epoch: 3 - score: 0.02326
 sToxic: 0.02381 - IsAbusive: 0.03509 - IsHatespeech: 0.00000
800/800 [==============================] - 2s 3ms/step - loss: 0.5728 - acc: 0.6929 - val_loss: 0.5644 - val_acc: 0.7200


...

/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 7/20
768/800 [===========================>..] - ETA: 0s - loss: 0.2453 - acc: 0.9141
 train: ROC-AUC - epoch: 7 - score: 0.99319
 IsToxic: 0.99734 - IsAbusive: 0.99906 - IsHatespeech: 0.97918
 train: F1 Score - epoch: 7 - score: 0.91972
 sToxic: 0.97668 - IsAbusive: 0.98929 - IsHatespeech: 0.30534

 val: ROC-AUC - epoch: 7 - score: 0.80416
 IsToxic: 0.75866 - IsAbusive: 0.79355 - IsHatespeech: 0.67673
 val: F1 Score - epoch: 7 - score: 0.59475
 sToxic: 0.68108 - IsAbusive: 0.60000 - IsHatespeech: 0.00000
800/800 [==============================] - 2s 3ms/step - loss: 0.2447 - acc: 0.9133 - val_loss: 0.4955 - val_acc: 0.7683
Epoch 8/20
768/800 [===========================>..] - ETA: 0s - loss: 0.1646 - acc: 0.9488
 train: ROC-AUC - epoch: 8 - score: 0.99867
 IsToxic: 0.99992 - IsAbusive: 0.99984 - IsHatespeech: 0.99699
 train: F1 Score - epoch: 8 - score: 0.94452
 sToxic: 0.99183 - IsAbusive: 0.99642 - IsHatespeech: 0.52000

 val: ROC-AUC - epoch: 8 - score: 0.80290
 IsToxic: 0.758

/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


768/800 [===========================>..] - ETA: 0s - loss: 0.1027 - acc: 0.9688
 train: ROC-AUC - epoch: 9 - score: 0.99970
 IsToxic: 0.99997 - IsAbusive: 0.99998 - IsHatespeech: 0.99814
 train: F1 Score - epoch: 9 - score: 0.98603
 sToxic: 0.99865 - IsAbusive: 0.99281 - IsHatespeech: 0.92233

 val: ROC-AUC - epoch: 9 - score: 0.79598
 IsToxic: 0.76570 - IsAbusive: 0.79646 - IsHatespeech: 0.69086
 val: F1 Score - epoch: 9 - score: 0.53750
 sToxic: 0.59756 - IsAbusive: 0.56000 - IsHatespeech: 0.12903
800/800 [==============================] - 2s 3ms/step - loss: 0.1025 - acc: 0.9692 - val_loss: 0.5741 - val_acc: 0.7533
Epoch 10/20
768/800 [===========================>..] - ETA: 0s - loss: 0.0696 - acc: 0.9878
 train: ROC-AUC - epoch: 10 - score: 0.99981
 IsToxic: 0.99999 - IsAbusive: 0.99999 - IsHatespeech: 0.99937
 train: F1 Score - epoch: 10 - score: 0.99006
 sToxic: 0.99865 - IsAbusive: 0.99642 - IsHatespeech: 0.94286

 val: ROC-AUC - epoch: 10 - score: 0.80277
 IsToxic: 0.77164 - Is

NameError: name 'e6_scores' is not defined

In [15]:
yt_scores_path = 'data/scores/youtoxic/youtoxic_{}'.format(time.time())
np.save(yt_scores_path, run_scores_matrix)

### Kaggle model on YouToxic data

In [6]:
X_youtoxic, Y_youtoxic = load_data_from_csv(path_youtoxic_data, youtoxic_features, youtoxic_classes)
X_tok_youtoxic = tokenize_sentences(X_youtoxic)

In [7]:
with open(embeddings_path, 'rb')  as file:
    embeddings_matrix = pickle.load(file)
    print(len(embeddings_matrix))

326175


In [8]:
X_input_youtoxic, max_comment_length = \
    convert_all_tokens_to_padded_indices(X_tok_youtoxic, embeddings_matrix)

maximum comment length: 969


In [13]:
loaded_model, _ = generate_model(max_comment_length=max_comment_length, classes=kaggle_classes, num_tokens=len(embeddings_matrix)+1)
loaded_model.load_weights(trained_weights)

Predict classes for youtoxic data set

In [14]:
predictions = loaded_model.predict(X_input_youtoxic)
roc_cl = roc_auc_score(Y_youtoxic.ix[:,0], predictions[:,0], average=None)
f1_cl = f1_score(Y_youtoxic.ix[:,0], predictions[:,0] >= 0.5, average=None)

/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [15]:
print('F1: ', f1_cl)
print('ROC: ', roc_cl)

F1:  [0.7810559  0.60393258]
ROC:  0.79664140073062


### Train YouToxic model from scratch

In [4]:
X_youtoxic, Y_youtoxic = load_data_from_csv(path_youtoxic_data, youtoxic_features, youtoxic_classes)
X_tok_youtoxic = tokenize_sentences(X_youtoxic)

In [5]:
with open(embeddings_path, 'rb')  as file:
    embeddings_matrix = pickle.load(file)
    print(len(embeddings_matrix))

326175


In [6]:
X_input_youtoxic, max_comment_length = \
    convert_all_tokens_to_padded_indices(X_tok_youtoxic, embeddings_matrix)

maximum comment length: 969


In [9]:
loaded_model, fc5 = generate_model(max_comment_length=max_comment_length, classes=kaggle_classes, num_tokens=len(embeddings_matrix)+1)

new_output = Dense(len(youtoxic_classes), activation='sigmoid')(fc5)
yt_model = Model(inputs=loaded_model.inputs, outputs=[new_output])

In [10]:
run_scores = []
for run in range(5):
    fold_scores = []
    kfold = KFold(5, shuffle=True)
    for train_index, test_index in kfold.split(X_input_youtoxic, Y_youtoxic):    
        X_train, X_test = X_input_youtoxic[train_index], X_input_youtoxic[test_index]
        y_train, y_test = Y_youtoxic.values[train_index], Y_youtoxic.values[test_index]
        scores = train_and_evaluate_model(yt_model, X_train, y_train, (X_test, y_test), \
                                 yt_epochs, batch_size, 'adam', 'binary_crossentropy', ['accuracy'], \
                                 seed=random_seed, runs=1, dataset=DataSet.YOUTOXIC)
        fold_scores.append(scores[0,:])
    fold_scores_matrix = np.array(fold_scores).mean(axis=0)
    run_scores.append(fold_scores_matrix)
run_scores_matrix = np.array(run_scores)

RUN 1/1
Train on 800 samples, validate on 200 samples
Epoch 1/20
768/800 [===========================>..] - ETA: 0s - loss: 0.6284 - acc: 0.6493
 train: ROC-AUC - epoch: 1 - score: 0.73794
 IsToxic: 0.85077 - IsAbusive: 0.71469 - IsHatespeech: 0.51157
 train: F1 Score - epoch: 1 - score: 0.00000
 sToxic: 0.00000 - IsAbusive: 0.00000 - IsHatespeech: 0.00000

 val: ROC-AUC - epoch: 1 - score: 0.67914
 IsToxic: 0.58641 - IsAbusive: 0.58512 - IsHatespeech: 0.44892
 val: F1 Score - epoch: 1 - score: 0.00000
 sToxic: 0.00000 - IsAbusive: 0.00000 - IsHatespeech: 0.00000
800/800 [==============================] - 7s 8ms/step - loss: 0.6291 - acc: 0.6496 - val_loss: 0.5934 - val_acc: 0.6733


/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'pre

Epoch 2/20
768/800 [===========================>..] - ETA: 0s - loss: 0.5802 - acc: 0.6884
 train: ROC-AUC - epoch: 2 - score: 0.78184
 IsToxic: 0.98597 - IsAbusive: 0.87798 - IsHatespeech: 0.63592
 train: F1 Score - epoch: 2 - score: 0.60947
 sToxic: 0.93425 - IsAbusive: 0.00000 - IsHatespeech: 0.00000

 val: ROC-AUC - epoch: 2 - score: 0.68753
 IsToxic: 0.61782 - IsAbusive: 0.60878 - IsHatespeech: 0.47500
 val: F1 Score - epoch: 2 - score: 0.32000
 sToxic: 0.50867 - IsAbusive: 0.00000 - IsHatespeech: 0.00000
800/800 [==============================] - 2s 3ms/step - loss: 0.5825 - acc: 0.6871 - val_loss: 0.5882 - val_acc: 0.6883


/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 3/20
768/800 [===========================>..] - ETA: 0s - loss: 0.5602 - acc: 0.7218
 train: ROC-AUC - epoch: 3 - score: 0.82264
 IsToxic: 0.99291 - IsAbusive: 0.95346 - IsHatespeech: 0.86649
 train: F1 Score - epoch: 3 - score: 0.05897
 sToxic: 0.08831 - IsAbusive: 0.04181 - IsHatespeech: 0.00000

 val: ROC-AUC - epoch: 3 - score: 0.70575
 IsToxic: 0.68657 - IsAbusive: 0.65186 - IsHatespeech: 0.53618
 val: F1 Score - epoch: 3 - score: 0.00000
 sToxic: 0.00000 - IsAbusive: 0.00000 - IsHatespeech: 0.00000
800/800 [==============================] - 2s 3ms/step - loss: 0.5642 - acc: 0.7171 - val_loss: 0.5826 - val_acc: 0.6733


...

Epoch 8/20
768/800 [===========================>..] - ETA: 0s - loss: 0.1595 - acc: 0.9358
 train: ROC-AUC - epoch: 8 - score: 0.99591
 IsToxic: 0.99964 - IsAbusive: 0.99953 - IsHatespeech: 0.99457
 train: F1 Score - epoch: 8 - score: 0.90741
 sToxic: 0.98343 - IsAbusive: 0.98252 - IsHatespeech: 0.00000

 val: ROC-AUC - epoch: 8 - score: 0.78871
 IsToxic: 0.75241 - IsAbusive: 0.80398 - IsHatespeech: 0.69716
 val: F1 Score - epoch: 8 - score: 0.55215
 sToxic: 0.61988 - IsAbusive: 0.59200 - IsHatespeech: 0.00000
800/800 [==============================] - 2s 3ms/step - loss: 0.1605 - acc: 0.9354 - val_loss: 0.5517 - val_acc: 0.7567


/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 9/20
768/800 [===========================>..] - ETA: 0s - loss: 0.1282 - acc: 0.9449
 train: ROC-AUC - epoch: 9 - score: 0.99862
 IsToxic: 0.99999 - IsAbusive: 0.99997 - IsHatespeech: 0.99964
 train: F1 Score - epoch: 9 - score: 0.92232
 sToxic: 0.99178 - IsAbusive: 0.98944 - IsHatespeech: 0.16949

 val: ROC-AUC - epoch: 9 - score: 0.78883
 IsToxic: 0.75070 - IsAbusive: 0.79932 - IsHatespeech: 0.68637
 val: F1 Score - epoch: 9 - score: 0.55090
 sToxic: 0.62570 - IsAbusive: 0.57600 - IsHatespeech: 0.00000
800/800 [==============================] - 2s 3ms/step - loss: 0.1248 - acc: 0.9467 - val_loss: 0.5885 - val_acc: 0.7500
Epoch 10/20
 64/800 [=>............................] - ETA: 1s - loss: 0.0663 - acc: 0.9844

/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


768/800 [===========================>..] - ETA: 0s - loss: 0.0960 - acc: 0.9622
 train: ROC-AUC - epoch: 10 - score: 0.99988
 IsToxic: 1.00000 - IsAbusive: 0.99999 - IsHatespeech: 0.99996
 train: F1 Score - epoch: 10 - score: 0.98457
 sToxic: 0.99728 - IsAbusive: 0.99822 - IsHatespeech: 0.89796

 val: ROC-AUC - epoch: 10 - score: 0.78285
 IsToxic: 0.74679 - IsAbusive: 0.79552 - IsHatespeech: 0.71853
 val: F1 Score - epoch: 10 - score: 0.57955
 sToxic: 0.66667 - IsAbusive: 0.56693 - IsHatespeech: 0.12121
800/800 [==============================] - 2s 3ms/step - loss: 0.0947 - acc: 0.9633 - val_loss: 0.6287 - val_acc: 0.7533
Epoch 11/20
768/800 [===========================>..] - ETA: 0s - loss: 0.0690 - acc: 0.9813
 train: ROC-AUC - epoch: 11 - score: 0.99998
 IsToxic: 1.00000 - IsAbusive: 1.00000 - IsHatespeech: 1.00000
 train: F1 Score - epoch: 11 - score: 0.98797
 sToxic: 1.00000 - IsAbusive: 0.99822 - IsHatespeech: 0.91457

 val: ROC-AUC - epoch: 11 - score: 0.78828
 IsToxic: 0.74247 

In [11]:
yt_scores_path = 'data/scores/youtoxic/youtoxic_fromscratch_{}'.format(time.time())
np.save(yt_scores_path, run_scores_matrix)

### Transfer learning with smaller learning rate

In [4]:
X_youtoxic, Y_youtoxic = load_data_from_csv(path_youtoxic_data, youtoxic_features, youtoxic_classes)

X_tok_youtoxic = tokenize_sentences(X_youtoxic)

In [5]:
with open(embeddings_path, 'rb')  as file:
    embeddings_matrix = pickle.load(file)
    print(len(embeddings_matrix))

326175


In [6]:
X_input_youtoxic, max_comment_length = \
    convert_all_tokens_to_padded_indices(X_tok_youtoxic, embeddings_matrix)

maximum comment length: 969


In [7]:
loaded_model, fc5 = generate_model(max_comment_length=max_comment_length, classes=kaggle_classes, num_tokens=len(embeddings_matrix)+1)
loaded_model.load_weights(trained_weights)

new_output = Dense(len(youtoxic_classes), activation='sigmoid')(fc5)
yt_model = Model(inputs=loaded_model.inputs, outputs=[new_output])

In [11]:
run_scores = []
for run in range(1):
    fold_scores = []
    kfold = KFold(5, shuffle=True)
    for train_index, test_index in kfold.split(X_input_youtoxic, Y_youtoxic):    
        optimizer = Adam(lr=0.0001)
        X_train, X_test = X_input_youtoxic[train_index], X_input_youtoxic[test_index]
        y_train, y_test = Y_youtoxic.values[train_index], Y_youtoxic.values[test_index]
        scores = train_and_evaluate_model(yt_model, X_train, y_train, (X_test, y_test), \
                                 100, batch_size, optimizer, 'binary_crossentropy', ['accuracy'], \
                                 seed=random_seed, runs=1, dataset=DataSet.YOUTOXIC)
        fold_scores.append(scores[0,:])
    fold_scores_matrix = np.array(fold_scores).mean(axis=0)
    run_scores.append(fold_scores_matrix)
run_scores_matrix = np.array(run_scores)

RUN 1/1
Train on 800 samples, validate on 200 samples
Epoch 1/100
768/800 [===========================>..] - ETA: 0s - loss: 0.6736 - acc: 0.6385
 train: ROC-AUC - epoch: 1 - score: 0.68704
 IsToxic: 0.58296 - IsAbusive: 0.60984 - IsHatespeech: 0.43627
 train: F1 Score - epoch: 1 - score: 0.37923
 sToxic: 0.55893 - IsAbusive: 0.00000 - IsHatespeech: 0.00000

 val: ROC-AUC - epoch: 1 - score: 0.66884
 IsToxic: 0.57186 - IsAbusive: 0.46737 - IsHatespeech: 0.48007
 val: F1 Score - epoch: 1 - score: 0.37419
 sToxic: 0.55502 - IsAbusive: 0.00000 - IsHatespeech: 0.00000
800/800 [==============================] - 3s 3ms/step - loss: 0.6732 - acc: 0.6404 - val_loss: 0.6596 - val_acc: 0.6767


/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 2/100
768/800 [===========================>..] - ETA: 0s - loss: 0.6464 - acc: 0.6623
 train: ROC-AUC - epoch: 2 - score: 0.70443
 IsToxic: 0.67358 - IsAbusive: 0.65976 - IsHatespeech: 0.44247
 train: F1 Score - epoch: 2 - score: 0.00786
 sToxic: 0.01609 - IsAbusive: 0.00000 - IsHatespeech: 0.00000

 val: ROC-AUC - epoch: 2 - score: 0.66710
 IsToxic: 0.56512 - IsAbusive: 0.46964 - IsHatespeech: 0.48007
 val: F1 Score - epoch: 2 - score: 0.00000
 sToxic: 0.00000 - IsAbusive: 0.00000 - IsHatespeech: 0.00000
800/800 [==============================] - 2s 3ms/step - loss: 0.6462 - acc: 0.6642 - val_loss: 0.6362 - val_acc: 0.6783


/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 3/100
768/800 [===========================>..] - ETA: 0s - loss: 0.6248 - acc: 0.6758
 train: ROC-AUC - epoch: 3 - score: 0.71561
 IsToxic: 0.73593 - IsAbusive: 0.67904 - IsHatespeech: 0.46225
 train: F1 Score - epoch: 3 - score: 0.00000
 sToxic: 0.00000 - IsAbusive: 0.00000 - IsHatespeech: 0.00000

 val: ROC-AUC - epoch: 3 - score: 0.66646
 IsToxic: 0.55374 - IsAbusive: 0.48053 - IsHatespeech: 0.48380
 val: F1 Score - epoch: 3 - score: 0.00000
 sToxic: 0.00000 - IsAbusive: 0.00000 - IsHatespeech: 0.00000
800/800 [==============================] - 2s 3ms/step - loss: 0.6244 - acc: 0.6775 - val_loss: 0.6162 - val_acc: 0.6783


...

/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


768/800 [===========================>..] - ETA: 0s - loss: 0.1305 - acc: 0.9631
 train: ROC-AUC - epoch: 55 - score: 0.99884
 IsToxic: 0.99997 - IsAbusive: 0.99944 - IsHatespeech: 0.99434
 train: F1 Score - epoch: 55 - score: 0.95623
 sToxic: 0.99598 - IsAbusive: 0.99296 - IsHatespeech: 0.65882

 val: ROC-AUC - epoch: 55 - score: 0.80364
 IsToxic: 0.75598 - IsAbusive: 0.79135 - IsHatespeech: 0.77628
 val: F1 Score - epoch: 55 - score: 0.55769
 sToxic: 0.65193 - IsAbusive: 0.50943 - IsHatespeech: 0.08000
800/800 [==============================] - 2s 3ms/step - loss: 0.1329 - acc: 0.9621 - val_loss: 0.4909 - val_acc: 0.7700
Epoch 56/100
768/800 [===========================>..] - ETA: 0s - loss: 0.1287 - acc: 0.9648
 train: ROC-AUC - epoch: 56 - score: 0.99911
 IsToxic: 0.99998 - IsAbusive: 0.99950 - IsHatespeech: 0.99471
 train: F1 Score - epoch: 56 - score: 0.96043
 sToxic: 0.99598 - IsAbusive: 0.99296 - IsHatespeech: 0.70455

 val: ROC-AUC - epoch: 56 - score: 0.80228
 IsToxic: 0.75699

Epoch 68/100
768/800 [===========================>..] - ETA: 0s - loss: 0.0727 - acc: 0.9874
 train: ROC-AUC - epoch: 68 - score: 0.99975
 IsToxic: 0.99999 - IsAbusive: 0.99998 - IsHatespeech: 0.99794
 train: F1 Score - epoch: 68 - score: 0.99284
 sToxic: 0.99867 - IsAbusive: 0.99824 - IsHatespeech: 0.95890

 val: ROC-AUC - epoch: 68 - score: 0.80422
 IsToxic: 0.75486 - IsAbusive: 0.79068 - IsHatespeech: 0.79972
 val: F1 Score - epoch: 68 - score: 0.55484
 sToxic: 0.63277 - IsAbusive: 0.53704 - IsHatespeech: 0.08000
800/800 [==============================] - 2s 3ms/step - loss: 0.0738 - acc: 0.9867 - val_loss: 0.5265 - val_acc: 0.7700
Epoch 69/100
768/800 [===========================>..] - ETA: 0s - loss: 0.0685 - acc: 0.9891
 train: ROC-AUC - epoch: 69 - score: 0.99976
 IsToxic: 0.99999 - IsAbusive: 0.99999 - IsHatespeech: 0.99798
 train: F1 Score - epoch: 69 - score: 0.99415
 sToxic: 0.99867 - IsAbusive: 0.99824 - IsHatespeech: 0.96833

 val: ROC-AUC - epoch: 69 - score: 0.80448
 IsT

Epoch 81/100
768/800 [===========================>..] - ETA: 0s - loss: 0.0406 - acc: 0.9961
 train: ROC-AUC - epoch: 81 - score: 0.99988
 IsToxic: 0.99999 - IsAbusive: 0.99999 - IsHatespeech: 0.99931
 train: F1 Score - epoch: 81 - score: 0.99806
 sToxic: 0.99867 - IsAbusive: 0.99824 - IsHatespeech: 0.99559

 val: ROC-AUC - epoch: 81 - score: 0.80553
 IsToxic: 0.75303 - IsAbusive: 0.78825 - IsHatespeech: 0.81250
 val: F1 Score - epoch: 81 - score: 0.55195
 sToxic: 0.62428 - IsAbusive: 0.55556 - IsHatespeech: 0.07407
800/800 [==============================] - 2s 3ms/step - loss: 0.0411 - acc: 0.9958 - val_loss: 0.5719 - val_acc: 0.7700
Epoch 82/100
768/800 [===========================>..] - ETA: 0s - loss: 0.0404 - acc: 0.9952
 train: ROC-AUC - epoch: 82 - score: 0.99992
 IsToxic: 0.99999 - IsAbusive: 0.99999 - IsHatespeech: 0.99960
 train: F1 Score - epoch: 82 - score: 0.99806
 sToxic: 0.99867 - IsAbusive: 0.99824 - IsHatespeech: 0.99559

 val: ROC-AUC - epoch: 82 - score: 0.80656
 IsT

Epoch 94/100
768/800 [===========================>..] - ETA: 0s - loss: 0.0245 - acc: 0.9978
 train: ROC-AUC - epoch: 94 - score: 0.99999
 IsToxic: 0.99999 - IsAbusive: 0.99999 - IsHatespeech: 0.99995
 train: F1 Score - epoch: 94 - score: 0.99806
 sToxic: 0.99867 - IsAbusive: 0.99824 - IsHatespeech: 0.99559

 val: ROC-AUC - epoch: 94 - score: 0.80577
 IsToxic: 0.74936 - IsAbusive: 0.78272 - IsHatespeech: 0.81416
 val: F1 Score - epoch: 94 - score: 0.55799
 sToxic: 0.62147 - IsAbusive: 0.54867 - IsHatespeech: 0.20690
800/800 [==============================] - 2s 3ms/step - loss: 0.0243 - acc: 0.9979 - val_loss: 0.6131 - val_acc: 0.7650
Epoch 95/100
768/800 [===========================>..] - ETA: 0s - loss: 0.0217 - acc: 0.9987
 train: ROC-AUC - epoch: 95 - score: 0.99999
 IsToxic: 0.99999 - IsAbusive: 0.99999 - IsHatespeech: 0.99995
 train: F1 Score - epoch: 95 - score: 0.99806
 sToxic: 0.99867 - IsAbusive: 0.99824 - IsHatespeech: 0.99559

 val: ROC-AUC - epoch: 95 - score: 0.80745
 IsT

In [12]:
yt_scores_path = 'data/scores/youtoxic/youtoxic_smalllr_{}'.format(time.time())
np.save(yt_scores_path, run_scores_matrix)